In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import stock_function as stock

%matplotlib inline
plt.style.use("fivethirtyeight")

In [4]:
#Load stock data
company = "TSLA"
start = dt.date(2022, 1, 1)
end = dt.date.today()

df = web.DataReader(company, "yahoo", start, end)
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-01-03,1201.069946,1136.040039,1147.750000,1199.780029,34643800,1199.780029
2022-01-04,1208.000000,1123.050049,1189.550049,1149.589966,33416100,1149.589966
2022-01-05,1170.339966,1081.010010,1146.650024,1088.119995,26706600,1088.119995
2022-01-06,1088.000000,1020.500000,1077.000000,1064.699951,30112200,1064.699951
2022-01-07,1080.930054,1010.000000,1080.369995,1026.959961,28054900,1026.959961
...,...,...,...,...,...,...
2022-03-25,1021.799988,997.320007,1008.000000,1010.640015,20642900,1010.640015
2022-03-28,1097.880005,1053.599976,1065.099976,1091.839966,34168700,1091.839966
2022-03-29,1114.770020,1073.109985,1107.989990,1099.569946,24538300,1099.569946


In [ ]:
#Visually show the stock price
plt.figure(figsize=(16, 4))
plt.plot(df['Close'], label='Close')
plt.title(f'{company} Close Price')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Price USD', fontsize=18)
plt.show()


In [ ]:
#Calculate the On Balance Volume (OBV)
OBV = []
OBV.append(0)

#Loop through the data set (close price) from the second row (index 1) to the end of the data set
for i in range(1, len(df.Close)):
    if df.Close[i] > df.Close[i-1]:
        OBV.append(OBV[-1] + df.Volume[i])
    elif df.Close[i] < df.Close[i-1]:
        OBV.append(OBV[-1] - df.Volume[i])
    else:
        OBV.append(OBV[-1])

In [ ]:
#Store the OBV and OBV Exponential Moving Average (EMA) into new columns
df['OBV'] = OBV
df['OBV_EMA'] = df['OBV'].ewm(span=20).mean()
#Show the data
df

In [ ]:
def add_obv_and_obv_ema(df):
    #Calculate the On Balance Volume (OBV)
    OBV = []
    OBV.append(0)

    #Loop through the data set (close price) from the second row (index 1) to the end of the data set
    for i in range(1, len(df.Close)):
        if df.Close[i] > df.Close[i-1]:
            OBV.append(OBV[-1] + df.Volume[i])
        elif df.Close[i] < df.Close[i-1]:
            OBV.append(OBV[-1] - df.Volume[i])
        else:
            OBV.append(OBV[-1])
    
    #Store the OBV and OBV Exponential Moving Average (EMA) into new columns
    df['OBV'] = OBV
    df['OBV_EMA'] = df['OBV'].ewm(span=20).mean()
    
    return df

In [ ]:
test_df = web.DataReader(company, "yahoo", start, end)
test_df = add_obv_and_obv_ema(test_df)
test_df

In [ ]:
#Plot the buy and sell prices
plt.figure(figsize=(16, 4))
plt.plot(df['OBV'], label='OBV', color ='orange')
plt.plot(df['OBV_EMA'], label='OBV_EMA', color='purple')
plt.title(f'{company} OBV / OBV EMA Chart')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Price USD', fontsize=18)
plt.legend()
plt.show()


In [ ]:
#Create a function to signal when to buy and sell the stock
# If OBV > OBV_EMA Then Buy
# IF OBV < OBV_EMA Then Sell
# Else Do Nothing

def obv_indicator(df, col1, col2):
    sigPriceBuy, sigPriceSell = [], []
    flag = -1
    #Loop through the length of the data set
    for i in range(0, len(df)):
        # If OBV > OBV_EMA Then Buy --> col1 => 'OBV' and col2 => 'OBV_EMA'
        if df[col1][i] > df[col2][i] and flag != 1:
            sigPriceBuy.append(df['Close'][i])
            sigPriceSell.append(np.nan)
            flag = 1
        # IF OBV < OBV_EMA Then Sell
        elif df[col1][i] < df[col2][i] and flag != 0:
            sigPriceSell.append(df['Close'][i])
            sigPriceBuy.append(np.nan)
            flag = 0
        else:
            sigPriceSell.append(np.nan)
            sigPriceBuy.append(np.nan)
            
    return (sigPriceBuy, sigPriceSell)

In [ ]:
#Create buy and sell columns
x = obv_indicator(df, 'OBV', 'OBV_EMA')
df['Buy_Signal_Price'] = x[0]
df['Sell_Signal_Price'] = x[1]
#Show the data set
df

In [ ]:
#Plot the buy and sell prices
plt.figure(figsize=(16, 4))
plt.plot(df['Close'], label='Close', alpha = 0.35)
plt.scatter(df.index, df['Buy_Signal_Price'], label='Buy Signal', marker = '^', alpha= 1, color='green')
plt.scatter(df.index, df['Sell_Signal_Price'], label='Sell Signal', marker = 'v', alpha=1, color='red')
plt.title(f'{company} Buy and Sell Signals')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Price USD', fontsize=18)
plt.legend()
plt.show()

In [10]:
fct_df = web.DataReader(company, "yahoo", start, end)
fct_df = stock.on_balance_volume(fct_df)
fct_df = stock.obv_indicator(fct_df, 'OBV', 'OBV_EMA')
fct_df

,High,Low,Open,Close,Volume,Adj Close,OBV,OBV_EMA,Buy_Signal_Price,Sell_Signal_Price
Date,,,,,,,,,,
2022-01-03,1201.069946,1136.040039,1147.750000,1199.780029,34643800,1199.780029,0,0.000000e+00,NaN,NaN
2022-01-04,1208.000000,1123.050049,1189.550049,1149.589966,33416100,1149.589966,-33416100,-1.754345e+07,NaN,1149.589966
2022-01-05,1170.339966,1081.010010,1146.650024,1088.119995,26706600,1088.119995,-60122700,-3.317830e+07,NaN,NaN
2022-01-06,1088.000000,1020.500000,1077.000000,1064.699951,30112200,1064.699951,-90234900,-4.964966e+07,NaN,NaN
2022-01-07,1080.930054,1010.000000,1080.369995,1026.959961,28054900,1026.959961,-118289800,-6.625312e+07,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-03-25,1021.799988,997.320007,1008.000000,1010.640015,20642900,1010.640015,-14447000,-1.166789e+08,NaN,NaN
2022-03-28,1097.880005,1053.599976,1065.099976,1091.839966,34168700,1091.839966,19721700,-1.036529e+08,NaN,NaN
2022-03-29,1114.770020,1073.109985,1107.989990,1099.569946,24538300,1099.569946,44260000,-8.953111e+07,NaN,NaN
